In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Latin_Macro_Watch_Dataset.csv")

<ipython-input-2-e9095707c131>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Latin_Macro_Watch_Dataset.csv")


## Variables

In [3]:
variables = df.groupby(["Frequency", "Indicator", "Unit", "Definition"]).count().reset_index()
variables.to_excel('LMV_VARS.xlsx')

<ipython-input-3-48e617480dfc>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  variables.to_excel('LMV_VARS.xlsx')


## Nans por país

In [48]:
df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].dt.year

In [67]:
writer = pd.ExcelWriter('LMV_RAW.xlsx')

for freq in df.Frequency.unique():
    anual = df[df.Frequency==freq]

    count_df = anual.groupby(["Country", "year"])["Value"].count().reset_index()
    count_pv = count_df.pivot_table(index='Country', columns='year', Values='Value', fill_value=0)

    nan_df = anual.groupby(["Country", "year"])["Value"].apply(lambda x: x.isna().sum()).reset_index()
    nan_pv = nan_df.pivot_table(index='Country', columns='year', values='Value', fill_value=0)

    total_pv = nan_pv + count_pv
    nan_pvp = nan_pv/count_pv
    porcentaje_nan = ((nan_pv/total_pv)*100)

    total_pv["N"] = "Total"
    nan_pv["N"] = "Nans"
    porcentaje_nan["N"] = "%Nans"

    dff = pd.concat([total_pv, nan_pv, porcentaje_nan], axis = 0)

    dff.to_excel(writer, sheet_name=f'{freq}')



<ipython-input-67-1571f9846545>:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  writer = pd.ExcelWriter('LMV_RAW.xlsx')


In [76]:
variables.to_excel(writer, sheet_name='variable_frequency')
writer.close()

## Indicadores

In [17]:
writer = pd.ExcelWriter('LMV_MEANS.xlsx')

for freq in df.Frequency.unique():
    dq = df[df.Frequency==freq]
    valores = dq.drop_duplicates(["Indicator", "Unit"])[["Area", "Topic", "Source", "Indicator", "Definition", "Unit"]].reset_index(drop=True).sort_values(["Indicator", "Unit"])

    dff = pd.DataFrame()

    for variable, unidad in zip(valores.Indicator, valores.Unit):
        dfk = dq[(df.Indicator == variable) & (dq.Unit == unidad)]
        dfk['Value'] = dfk['Value'].str.replace(',', '').astype(float)

        dfv = dfk.groupby("Country")["Value"].mean().reset_index()
        dfv.columns = ["Country", variable]

        stats = {
            'min': dfk['Value'].min(),
            'max': dfk['Value'].max(),
            'mean': dfk['Value'].mean(),
            'p50': dfk['Value'].quantile(0.50),
            'std': dfk['Value'].std(),
            'nans': dfk['Value'].isna().sum(),
            'N': len(dfk['Value'])
        }

        # Convertir las estadísticas en un DataFrame temporal
        stats_df = pd.DataFrame(stats, index=['stats'])

        stats_df = stats_df.T.reset_index()
        stats_df.columns = ["Country", variable]

        # Concatenar el DataFrame original con el nuevo DataFrame de estadísticas
        dfv = pd.concat([dfv, stats_df])
        dfv.index = dfv.Country
        dfv = dfv[[variable]]

        dff = pd.concat([dff, dfv], axis = 1)

    dff.to_excel(writer, sheet_name=f'{freq}')

writer.close()

<ipython-input-17-e9390f1e3ed2>:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  writer = pd.ExcelWriter('LMV_MEANS.xlsx')
<ipython-input-17-e9390f1e3ed2>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfk = dq[(df.Indicator == variable) & (dq.Unit == unidad)]
<ipython-input-17-e9390f1e3ed2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfk['Value'] = dfk['Value'].str.replace(',', '').astype(float)
<ipython-input-17-e9390f1e3ed2>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfk = dq[(df.Indicator == variable) & (dq.Unit == unidad)]
<ipython-input-17-e9390f1e3ed2>:11: SettingWithCopyWarning: 
A value is tryin

In [18]:
writer = pd.ExcelWriter('LMV_VALORES_FRECUENCIAS.xlsx')

for freq in df.Frequency.unique():
    dq = df[df.Frequency==freq]
    valores = dq.drop_duplicates(["Indicator", "Unit"])[["Area", "Topic", "Source", "Indicator", "Definition", "Unit"]].reset_index(drop=True).sort_values(["Indicator", "Unit"])

    # dff = pd.DataFrame()

    # for variable, unidad in zip(valores.Indicator, valores.Unit):
    #     dfk = dq[(df.Indicator == variable) & (dq.Unit == unidad)]
    #     dfk['Value'] = dfk['Value'].str.replace(',', '').astype(float)

    #     dfv = dfk.groupby("Country")["Value"].mean().reset_index()
    #     dfv.columns = ["Country", variable]

    #     stats = {
    #         'min': dfk['Value'].min(),
    #         'max': dfk['Value'].max(),
    #         'mean': dfk['Value'].mean(),
    #         'p50': dfk['Value'].quantile(0.50),
    #         'std': dfk['Value'].std(),
    #         'nans': dfk['Value'].isna().sum(),
    #         'N': len(dfk['Value'])
    #     }

    #     # Convertir las estadísticas en un DataFrame temporal
    #     stats_df = pd.DataFrame(stats, index=['stats'])

    #     stats_df = stats_df.T.reset_index()
    #     stats_df.columns = ["Country", variable]

    #     # Concatenar el DataFrame original con el nuevo DataFrame de estadísticas
    #     dfv = pd.concat([dfv, stats_df])
    #     dfv.index = dfv.Country
    #     dfv = dfv[[variable]]

    #     dff = pd.concat([dff, dfv], axis = 1)

    valores.to_excel(writer, sheet_name=f'{freq}')

writer.close()

<ipython-input-18-2704f0351127>:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  writer = pd.ExcelWriter('LMV_VALORES_FRECUENCIAS.xlsx')
